In [ ]:
import pandas as pd

In [ ]:
nsw_co2 = pd.read_csv('./emissions/nsw_co2intensity.csv')
vic_co2 = pd.read_csv('./emissions/vic_co2intensity.csv')

In [ ]:
vm_skus_tdp = pd.read_csv('./azurevms/azurevmskus_withtdp.csv')
vm_skus_tdp

In [ ]:
# These are the inputs

desired_sku_list = [{'sku':'Standard_D3_v2','count':4}, {'sku':'Standard_F16s_v2', 'count':2}, {'sku':'Standard_NC12','count': 1}]
default_hours_month = 730.5
start_month = 10
average_util_pct = .30
desired_region = 'vic'
microsoft_pue_constant = 1.125

In [ ]:
if(desired_region == 'nsw'):
    carbonintensity = nsw_co2[['date','CarbonIntensity']]
else:
    carbonintensity = vic_co2[['date','CarbonIntensity']]



In [ ]:
# Filter skus
skus_list = [x['sku'] for x in desired_sku_list]
is_in_desired_skus = vm_skus_tdp['skuname'].isin(skus_list)

vm_desired_skus = vm_skus_tdp[is_in_desired_skus]
vm_desired_skus

In [ ]:
# Append SKU count
vm_desired_skus['skucount'] =  vm_desired_skus['skuname'].map(lambda x: [y['count'] for y in desired_sku_list if y['sku'] == x].pop() )
vm_desired_skus['skucount']

In [ ]:
vm_desired_skus

In [ ]:
vm_desired_skus['totalmonthlykwh'] = vm_desired_skus['totaltdp'] * vm_desired_skus['skucount'] * default_hours_month / 1000 * microsoft_pue_constant * average_util_pct
vm_desired_skus['totalmonthlykwh']

In [ ]:
carbonintensity['CarbonIntensity']

In [ ]:
# Generate CO2eq emissions per month
desired_skus_with_monthly_co2 = vm_desired_skus.apply(lambda x: carbonintensity['CarbonIntensity'].map(lambda y: y * x['totalmonthlykwh']), axis=1)

In [ ]:
# Convert to KG
desired_skus_with_monthly_co2 = desired_skus_with_monthly_co2.apply(lambda x: x/1000)

In [ ]:
desired_skus_with_monthly_co2

desired_skus_with_monthly_co2['TotalYearlyCO2InKg'] = desired_skus_with_monthly_co2.sum(axis=1)
desired_skus_with_monthly_co2['TotalYearlyCO2InKg']

In [ ]:
desired_skus_with_monthly_co2

In [ ]:
vm_desired_skus = vm_desired_skus.join(desired_skus_with_monthly_co2)

In [ ]:
vm_desired_skus.head(5)